# Egobox

## Installation

You can install with the `pip` command as follows: 

```bash
pip install egobox
```

## Import

In [1]:
import numpy as np
import egobox as egx

# To display optimization information (none by default)
import logging
logging.basicConfig(level=logging.INFO)

## Continuous test functions 

In [2]:
xspecs_xsinx = egx.to_specs([[0., 25.]])
n_cstr_xsinx = 0

def xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    return y

In [3]:
xspecs_g24 = egx.to_specs([[0., 3.], [0., 4.]])
n_cstr_g24 = 2

# Objective
def G24(point):
    """
    Function g24
    1 global optimum y_opt = -5.5080 at x_opt =(2.3295, 3.1785)
    """
    p = np.atleast_2d(point)
    return - p[:, 0] - p[:, 1]

# Constraints < 0
def G24_c1(point):
    p = np.atleast_2d(point)
    return (- 2.0 * p[:, 0] ** 4.0
            + 8.0 * p[:, 0] ** 3.0 
            - 8.0 * p[:, 0] ** 2.0 
            + p[:, 1] - 2.0)

def G24_c2(point):
    p = np.atleast_2d(point)
    return (-4.0 * p[:, 0] ** 4.0
            + 32.0 * p[:, 0] ** 3.0
            - 88.0 * p[:, 0] ** 2.0
            + 96.0 * p[:, 0]
            + p[:, 1] - 36.0)

# Grouped evaluation
def g24(point):
    p = np.atleast_2d(point)
    return np.array([G24(p), G24_c1(p), G24_c2(p)]).T


## Ego optimizer written in Rust (a.k.a Egor)

In [14]:
egor = egx.Optimizer(g24, xspecs_g24, 
                     n_doe=10, 
                     n_cstr=n_cstr_g24, 
                     cstr_tol=1e-3,
                     infill_strategy=egx.InfillStrategy.WB2,
                     # expected=egx.ExpectedOptimum(val=-5.50, tol=1e-2),
                     # outdir="./out",
                     # hot_start=True
                    )  # see help(egor) for options

# Restrict regression and correlation models used
#egor = egx.Optimizer(g24, xlimits_g24, n_cstr=n_cstr_g24, n_doe=10,
#                      regr_spec=egx.RegressionSpec.LINEAR,
#                      corr_spec=egx.CorrelationSpec.MATERN32 | egx.CorrelationSpec.MATERN52)  

In [15]:
res = egor.minimize(n_eval=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")

INFO:egobox_ego.egor:Compute initial LHS with 10 points
INFO:egobox_moe.algorithm:Best expert Constant_Matern52 accuracy=0.000003676964964904883
INFO:egobox_moe.algorithm:Best expert -> Constant_Matern52
INFO:egobox_moe.algorithm:Best expert Constant_SquaredExponential accuracy=1.72466844480517
INFO:egobox_moe.algorithm:Best expert -> Constant_SquaredExponential
INFO:egobox_moe.algorithm:Best expert Constant_Matern52 accuracy=2.337683164462205
INFO:egobox_moe.algorithm:Best expert -> Constant_Matern52
INFO:egobox_ego.egor:Add 1 point:
INFO:egobox_ego.egor:  [[2.1207335084665533, 3.999999999678943]], shape=[1, 2], strides=[2, 1], layout=CFcf (0xf), const ndim=2
INFO:egobox_ego.egor:End iteration 1/20: Best fun(x)=[-2.4529407582748757, -1.565312711714646, -3.561112994676918] at x=[2.0161366008313566, 0.436804157443519]
INFO:egobox_moe.algorithm:Best expert Constant_Matern52 accuracy=0.0000029827852626507222
INFO:egobox_moe.algorithm:Best expert -> Constant_Matern52
INFO:egobox_moe.algori

Optimization f=[-5.508589493095267, 0.0005807745848427359, 0.0005743833708038437] at [2.329519700663474, 3.1790697924317923]


## Mixed integer test function

In [6]:
xspecs_mixint_xsinx = [egx.Vspec(egx.Vtype(egx.Vtype.INT), [0, 25])]
n_cstr_mixint_xsinx = 0

def mixint_xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    if (np.abs(np.linalg.norm(np.floor(x))-np.linalg.norm(x))< 1e-8):
        y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    else:
        raise ValueError(f"Bad input: mixint_xsinx accepts integer only, got {x}")
    print(f"obj={y} at {x}")
    return y

In [7]:
egor = egx.Optimizer(mixint_xsinx, xspecs_mixint_xsinx, 
                     n_doe=3, 
                     infill_strategy=egx.InfillStrategy.EI,
                     expected=egx.ExpectedOptimum(val=-15.12, tol=1e-2),
                    )  # see help(egor) for options
res = egor.minimize(n_eval=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")

INFO:egobox_ego.egor:Compute initial LHS with 3 points
INFO:egobox_moe.algorithm:Best expert Constant_SquaredExponential accuracy=9.841369315617833
INFO:egobox_moe.algorithm:Best expert -> Constant_SquaredExponential
INFO:egobox_ego.egor:Add 1 point:
INFO:egobox_ego.egor:  [[25.0]], shape=[1, 1], strides=[1, 1], layout=CFcf (0xf), const ndim=2
INFO:egobox_ego.egor:End iteration 1/27: Best fun(x)=[-14.1545328776705] at x=[19.547800735233757]
INFO:egobox_moe.algorithm:Best expert Constant_AbsoluteExponential accuracy=13.083582695169605
INFO:egobox_moe.algorithm:Best expert -> Constant_AbsoluteExponential


obj=[[ -9.28531859]
 [  3.14127616]
 [-14.15453288]] at [[16.]
 [ 7.]
 [20.]]
obj=[[11.42919546]] at [[25.]]


INFO:egobox_ego.egor:Add 1 point:
INFO:egobox_ego.egor:  [[19.144821836732]], shape=[1, 1], strides=[1, 1], layout=CFcf (0xf), const ndim=2
INFO:egobox_ego.egor:End iteration 2/27: Best fun(x)=[-15.121611536143384] at x=[19.144821836732]
INFO:egobox_ego.egor:Expected optimum : ApproxValue { value: -15.12, tolerance: 0.01 }
INFO:egobox_ego.egor:Expected optimum reached!
INFO:egobox_ego.egor:[[15.591052521407839, -9.285318589373164],
 [6.917934385007657, 3.14127615863859],
 [19.547800735233757, -14.1545328776705],
 [25.0, 11.429195456150415],
 [19.144821836732, -15.121611536143384]], shape=[5, 2], strides=[1, 5], layout=Ff (0xa), const ndim=2


obj=[[-15.12161154]] at [[19.]]
Optimization f=[-15.121611536143384] at [19.0]


## Usage

In [8]:
help(egor)

Help on Optimizer in module builtins object:

class Optimizer(object)
 |  Optimizer(fun, n_cstr=0, cstr_tol=1e-06, n_start=20, n_doe=0, regression_spec=7, correlation_spec=15, infill_strategy=1, n_parallel=1, par_infill_strategy=1, infill_optimizer=1, n_clusters=1)
 |  
 |  Optimizer constructor
 |  
 |  Parameters
 |  
 |      fun: array[n, nx]) -> array[n, ny]
 |          the function to be minimized
 |          fun(x) = [obj(x), cstr_1(x), ... cstr_k(x)] where
 |             obj is the objective function [n, nx] -> [n, 1]
 |             cstr_i is the ith constraint function [n, nx] -> [n, 1]
 |             an k the number of constraints (n_cstr)
 |             hence ny = 1 (obj) + k (cstrs)
 |          cstr functions are expected be negative (<=0) at the optimum.
 |  
 |      n_cstr (int):
 |          the number of constraint functions.
 |  
 |      cstr_tol (float):
 |          tolerance on constraints violation (cstr < tol).
 |  
 |      xspecs (list(Vspec)) where Vspec(vtype=FLOA